This will plot and compare the resulting data. To run this, make sure that you have already generated all data you want to be compared. If you want to run a sequential sort or the multiprocessing merge sort, run with run.py. If you want to run the MPI merge sort, run with mpi_run.py. 

In [4]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from cloudmesh.common.util import banner
from analysis import get_data
from analysis import read_log, read_logs
from analysis import generate_average

In [5]:
user = "alex"
node = "2400MHz"
sorts = ["seq-merge", "mp-mergesort"]
sort = "mp-mergesort"

sizes = [100, 1000, 10e4, 10e5]
sizes = [int(size) for size in sizes]

In [ ]:
# this will take a long time. 
# also if you don't want to use all your processors then use different commands. 
for sort in sorts:
    for size in sizes:
        run_cmd = f"./run.py --user={user} --node={node} --size={size} --sort={sort}"
        banner(run_cmd)
        os.system(run_cmd)

In [17]:
def plot_benchmark_by_size(df, size=None, tag=None, files=None, x="sizes", y="time"):
    names = "-".join(files)
    sns.lineplot(data=df, x=x, y=y, hue="name");
    t = tag.title()
    label = f"{t}".replace("_", " ")
    plt.title(f"{label}")
    if "time" in y: y = f"{y}/s"
    if "time" in x: x = f"{x}/s"
    plt.xlabel(x.capitalize())
    plt.ylabel(y.capitalize())
    plt.savefig(f"images/{tag}-{names}.png")
    plt.savefig(f"images/{tag}-{names}.pdf")

In [ ]:
files = ["alex"]
sorts = ["seq-merge", "mp-mergesort"]

frames = []
for file in files:
    for sort in sorts:
        frame = []
        for size in sizes: 
            size = int(size)
            log = f'{sort}-{node}-{file}'
            _frame = read_log(log, size=size, tag=sort)
            frame = frame + _frame
        frames.append(frame)
# print(frames)


df = pd.DataFrame()
for frame in frames:
    _df = pd.DataFrame(data=frame,
                columns=["processes", "time", "size", "name", "tag"])
    df = pd.concat([df, _df], ignore_index=True)
# plot_benchmark_by_size(df, "name", tag=sort, files=files, x="size", y="time")

In [9]:
df

,processes,time,size,name,tag
0,1,0.000,100,alex,seq-merge
1,1,0.000,100,alex,seq-merge
2,1,0.000,100,alex,seq-merge
3,1,0.000,100,alex,seq-merge
4,1,0.000,100,alex,seq-merge
...,...,...,...,...,...
515,1,5.948,1000000,alex,mp-mergesort
516,1,5.995,1000000,alex,mp-mergesort
517,1,6.042,1000000,alex,mp-mergesort
518,1,6.014,1000000,alex,mp-mergesort


In [22]:
df = df.pivot_table('time', ['tag', 'name', 'processes'], 'size')


KeyError: 'time'